In [ ]:
"""Importing libraries and reading in the test data"""
import mne
import pandas as pd
import numpy as np
from statistics import mean
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import matplotlib.pyplot as plt
import pywt
from entropy import*
from scipy.stats import entropy
import math

In [ ]:
"""

THE WAY THIS NOTEBOOK IS LAYED OUT IS AS FOLLOWS:

1ST: THE TRAINING DATA IS LOADED AND COMBINED INTO 1 DATAFRAME

2ND: CWT IS PERFORMED AND THEN COEFFICIENTS PLOTTED.

3RD: MOVING WINDOW AVERAGE PERMUTATION ENTROPY FOR THE COEFFICIENCTS IS CALCUATED AND PLOTTED.

4TH: MOVING WINDOW AVERAGE HIGUCHI FRACTAL DIMENSION FOR THE COEFFICIENCTS IS CALCUATED AND PLOTTED.

5TH: OPTIMAL FEATURES FROM SUB-BAND COEFFICIENTS ARE SELECTED AND JOINED IN A DATAFRAME

6TH: TRAINING DATA IS PASSED TO CV LASSO REGRESSION MODEL

7TH: TEST DATA IS LOADED AND JOINED

8TH: TEST DATA PASSED TO TRAINED MODEL

9TH: MODEL PERFORMANCE EVALUATED WITH MSE,RMSE,MAE

"""

In [ ]:
t1 = pd.read_excel('/Users/colin/Desktop/UNI STUFF/MSC8002 Thesis PT 2/Train.xlsx', sheet_name = 0)

In [ ]:
"""SIGNAL 1 TRANSFORM, EXTRACTION, WRITTEN TO XSXL FOR CH2 & X"""

In [ ]:
"""Setting T1 CH2 & X signals to numpy array"""
ch2 = t1.CH2.to_numpy()
sx = t1.X.to_numpy()

In [ ]:
"""Printing available wavelets"""
wavlist = pywt.wavelist(kind='continuous')
print(wavlist)

In [ ]:
"""Performing CWT on CH2 signal with 6 scales and using the Mexican Hat wavelet"""
#t = np.linspace(0,10,451584)
# Start = 0 
# End = 10 
# Samples to generate 

t1ch2signal = ch2
t1ch2scales = np.arange(1,6) #of scales


t1coefs, t1freqs = pywt.cwt(t1ch2signal, t1ch2scales, 'mexh')
# coefs -> stores all of the coefficients
# freqs -> stores all of the wavelet frequencies

In [ ]:
"""Subplots of Continuous Wavelet Transform coefficients"""

fig1 = make_subplots(rows = 2, cols = 3, start_cell = 'top-left', subplot_titles=("coef1", "coef2", "coef3", "coef4", "coef5", "Original Signal"))
fig1.append_trace(go.Line(y = t1coefs[0], name = 'coef1'), 1,1)
fig1.append_trace(go.Line(y = t1coefs[1], name = 'coef2'), 1,2)
fig1.append_trace(go.Line(y = t1coefs[2], name = 'coef3'), 1,3)
fig1.append_trace(go.Line(y = t1coefs[3], name = 'coef4'), 2,1)
fig1.append_trace(go.Line(y = t1coefs[4], name = 'coef5'), 2,2)
fig1.append_trace(go.Line(y = t1.CH2, name = 'Original Ch2 Signal'), 2,3)
fig1.update_layout(title="Continuous Wavelet Transform t1 CH2 Coefficients")

fig1.show()

In [ ]:
"""CALCULATING THE MOVING WINDOW AVERAGE PERMUTATION ENTROPY FOR T1 CH2 CWT SUB-BANDS"""

In [ ]:
""" coef1 permutation entropy"""
t1coef1PE = []
for i in range(0,len(t1coefs[0]), 128):
    t1coef1PE.append(perm_entropy(t1coefs[0][i:i+128], normalize=True))

In [ ]:
"""Moving Window Average for coef1 Permutation Entropy"""
t1avg_coef1PE = []
for i in range(0, len(t1coef1PE)-56, 1):
    t1avg_coef1PE.append(mean(t1coef1PE[i:i+56]))

In [ ]:
""" coef2 permutation entropy"""
t1coef2PE = []
for i in range(0,len(t1coefs[1]), 128):
    t1coef2PE.append(perm_entropy(t1coefs[1][i:i+128], normalize=True))

In [ ]:
"""Moving Window Average for coef2 Permutation Entropy"""
t1avg_coef2PE = []
for i in range(0, len(t1coef2PE)-56, 1):
    t1avg_coef2PE.append(mean(t1coef2PE[i:i+56]))

In [ ]:
""" coef3 permutation entropy"""
t1coef3PE = []
for i in range(0,len(t1coefs[2]), 128):
    t1coef3PE.append(perm_entropy(t1coefs[2][i:i+128], normalize=True))

In [ ]:
"""Moving Window Average for coef3 Permutation Entropy"""
t1avg_coef3PE = []
for i in range(0, len(t1coef3PE)-56, 1):
    t1avg_coef3PE.append(mean(t1coef3PE[i:i+56]))

In [ ]:
""" coef4 permutation entropy"""
t1coef4PE = []
for i in range(0,len(t1coefs[3]), 128):
    t1coef4PE.append(perm_entropy(t1coefs[3][i:i+128], normalize=True))

In [ ]:
"""Moving Window Average for cD4 Permutation Entropy"""
t1avg_coef4PE = []
for i in range(0, len(t1coef4PE)-56, 1):
    t1avg_coef4PE.append(mean(t1coef4PE[i:i+56]))

In [ ]:
""" coef5 permutation entropy"""
t1coef5PE = []
for i in range(0,len(t1coefs[4]), 128):
    t1coef5PE.append(perm_entropy(t1coefs[4][i:i+128], normalize=True))

In [ ]:
"""Moving Window Average for cD5 Permutation Entropy"""
t1avg_coef5PE = []
for i in range(0, len(t1coef5PE)-56, 1):
    t1avg_coef5PE.append(mean(t1coef5PE[i:i+56]))

In [ ]:
"""Matrix of Continuous Wavelet Transform coefficients Permutation Entropy"""

fig2 = make_subplots(rows = 2, cols = 3, start_cell = 'top-left', subplot_titles=("Avg coef1 PE","Avg coef2 PE","Avg coef3 PE","Avg coef4 PE","Avg coef5 PE", "BIS"))
fig2.append_trace(go.Line(y = t1avg_coef1PE, name = 'Coef1 PE'), 1,1)
fig2.append_trace(go.Line(y = t1avg_coef2PE, name = 'Coef2 PE'), 1,2)
fig2.append_trace(go.Line(y = t1avg_coef3PE, name = 'Coef3 PE'), 1,3)
fig2.append_trace(go.Line(y = t1avg_coef4PE, name = 'Coef4 PE'), 2,1)
fig2.append_trace(go.Line(y = t1avg_coef5PE, name = 'Coef5 PE'), 2,2)
fig2.append_trace(go.Line(y = t1.BIS[0:3535:], name = 'BIS'), 2,3)

fig2.update_layout(title="Continuous Wavelet Transform t1 CH2 Coefficients 56 Sec Window Average Perm Entropy")

fig2.show()

In [ ]:
"""CALCULATING THE MOVING WINDOW AVERAGE Higuchi Fractal Dimension FOR THE CH2 CWT SUB-BANDS"""

In [ ]:
"""coef1 Higuchi Fractal Dimension"""
t1coef1HFD = []
for i in range(0,len(t1coefs[0]), 128):
    t1coef1HFD.append(higuchi_fd(t1coefs[0][i:i+128], kmax = 10))

In [ ]:
"""Moving Window Average for coef1 Higuchi Fractal Dimension"""
t1avg_coef1HFD = []
for i in range(0, len(t1coef1HFD)-56, 1):
    t1avg_coef1HFD.append(mean(t1coef1HFD[i:i+56]))

In [ ]:
"""coef2 Higuchi Fractal Dimension"""
t1coef2HFD = []
for i in range(0,len(t1coefs[1]), 128):
    t1coef2HFD.append(higuchi_fd(t1coefs[1][i:i+128], kmax = 10))

In [ ]:
"""Moving Window Average for coef2 Higuchi Fractal Dimension"""
t1avg_coef2HFD = []
for i in range(0, len(t1coef2HFD)-56, 1):
    t1avg_coef2HFD.append(mean(t1coef2HFD[i:i+56]))

In [ ]:
"""coef3 Higuchi Fractal Dimension"""
t1coef3HFD = []
for i in range(0,len(t1coefs[2]), 128):
    t1coef3HFD.append(higuchi_fd(t1coefs[2][i:i+128], kmax = 10))

In [ ]:
"""Moving Window Average for coef3 Higuchi Fractal Dimension"""
t1avg_coef3HFD = []
for i in range(0, len(t1coef3HFD)-56, 1):
    t1avg_coef3HFD.append(mean(t1coef3HFD[i:i+56]))

In [ ]:
"""coef4 Higuchi Fractal Dimension"""
t1coef4HFD = []
for i in range(0,len(t1coefs[3]), 128):
    t1coef4HFD.append(higuchi_fd(t1coefs[3][i:i+128], kmax = 10))

In [ ]:
"""Moving Window Average for coef4 Higuchi Fractal Dimension"""
t1avg_coef4HFD = []
for i in range(0, len(t1coef4HFD)-56, 1):
    t1avg_coef4HFD.append(mean(t1coef4HFD[i:i+56]))

In [ ]:
"""coef5 Higuchi Fractal Dimension"""
t1coef5HFD = []
for i in range(0,len(t1coefs[4]), 128):
    t1coef5HFD.append(higuchi_fd(t1coefs[4][i:i+128], kmax = 10))

In [ ]:
"""Moving Window Average for coef5 Higuchi Fractal Dimension"""
t1avg_coef5HFD = []
for i in range(0, len(t1coef5HFD)-56, 1):
    t1avg_coef5HFD.append(mean(t1coef5HFD[i:i+56]))

In [ ]:
"""Matrix of Continuous Wavelet Transform CH2 coefficients Higuchi Fractal Dimension"""

fig3 = make_subplots(rows = 2, cols = 3, start_cell = 'top-left', subplot_titles=("Avg coef1 HFD","Avg coef2 HFD","Avg coef3 HFD","Avg coef4 HFD","Avg coef5 HFD", "BIS"))
fig3.append_trace(go.Line(y = t1avg_coef1HFD, name = 'Coef1 HFD'), 1,1)
fig3.append_trace(go.Line(y = t1avg_coef2HFD, name = 'Coef2 HFD'), 1,2)
fig3.append_trace(go.Line(y = t1avg_coef3HFD, name = 'Coef3 HFD'), 1,3)
fig3.append_trace(go.Line(y = t1avg_coef4HFD, name = 'Coef4 HFD'), 2,1)
fig3.append_trace(go.Line(y = t1avg_coef5HFD, name = 'Coef5 HFD'), 2,2)
fig3.append_trace(go.Line(y = t1.BIS[0:3535:], name = 'BIS'), 2,3)

fig3.update_layout(title="Continuous Wavelet Transform t1 CH2 Coefficients 56 Sec Window Average Higuchi Fractal Dimension")

fig3.show()

In [ ]:
"""COEF 3&4 SAMPLE ENTROPY"""

"""coef3 sample entropy"""
coef3SampE = []
for i in range(0,len(t1coefs[2]), 128):
    coef3SampE.append(sample_entropy(t1coefs[2][i:i+128], metric='chebyshev'))
    
"""Moving Window Average for coef3 Sample Entropy"""
avg_coef3SampE = []
for i in range(0, len(coef3SampE)-56, 1):
    avg_coef3SampE.append(mean(coef3SampE[i:i+56]))
    
"""coef4 sample entropy"""
coef4SampE = []
for i in range(0,len(t1coefs[3]), 128):
    coef4SampE.append(sample_entropy(t1coefs[3][i:i+128], metric='chebyshev'))
    
"""Moving Window Average for coef4 Sample Entropy"""
avg_coef4SampE = []
for i in range(0, len(coef4SampE)-56, 1):
    avg_coef4SampE.append(mean(coef4SampE[i:i+56]))

In [ ]:
"COEF 3&4 SPECTRAL ENTROPY"

"""coef3 Spectral entropy"""
"""User warning is only based on library using 2-second epochs"""
coef3SpecE = []
for i in range(0,len(t1coefs[2]), 128):
    coef3SpecE.append(spectral_entropy(t1coefs[2][i:i+128], sf = 128,method = 'welch', normalize=True))
    
"""Moving Window Average for coef3 Spectral Entropy"""
avg_coef3SpecE = []
for i in range(0, len(coef3SpecE)-56, 1):
    avg_coef3SpecE.append(mean(coef3SpecE[i:i+56]))
    
"""coef4 Spectral entropy"""
"""User warning is only based on library using 2-second epochs"""
coef4SpecE = []
for i in range(0,len(t1coefs[3]), 128):
    coef4SpecE.append(spectral_entropy(t1coefs[3][i:i+128], sf = 128,method = 'welch', normalize=True))
    
"""Moving Window Average for coef4 Spectral Entropy"""
avg_coef4SpecE = []
for i in range(0, len(coef4SpecE)-56, 1):
    avg_coef4SpecE.append(mean(coef4SpecE[i:i+56]))

In [ ]:
"""WRITING THE NEW FEATURES TO XLSX"""
new_features = pd.DataFrame(avg_coef4SampE)
new_features['c4SampE'] = avg_coef4SpecE
new_features['c3SampE'] = avg_coef3SampE
new_features['c3SpecE'] = avg_coef3SpecE

new_features.rename(columns = {0:'c4SampE'}, inplace = True)

new_features.head()

new_features.to_excel("New1.xlsx", sheet_name='Case1', engine = 'xlsxwriter')

In [ ]:
"""STOP HERE"""
from

In [ ]:
"""CWT AND EXTRACTION FOR X"""

In [ ]:
"""CWT FOR X CHANNEL"""
t1Xsignal = sx
t1xscales = np.arange(1,6) #of scales


t1xcoefs, t1xfreqs = pywt.cwt(t1Xsignal, t1xscales, 'mexh')
# coefs -> stores all of the coefficients
# freqs -> stores all of the wavelet frequencies

In [ ]:
"""Subplots of Continuous Wavelet Transform coefficients"""

fig4 = make_subplots(rows = 2, cols = 3, start_cell = 'top-left', subplot_titles=("coef1", "coef2", "coef3", "coef4", "coef5", "Original Signal"))
fig4.append_trace(go.Line(y = t1xcoefs[0], name = 'coef1'), 1,1)
fig4.append_trace(go.Line(y = t1xcoefs[1], name = 'coef2'), 1,2)
fig4.append_trace(go.Line(y = t1xcoefs[2], name = 'coef3'), 1,3)
fig4.append_trace(go.Line(y = t1xcoefs[3], name = 'coef4'), 2,1)
fig4.append_trace(go.Line(y = t1xcoefs[4], name = 'coef5'), 2,2)
fig4.append_trace(go.Line(y = t1.X, name = 'Original Ch2 Signal'), 2,3)
fig4.update_layout(title="Continuous Wavelet Transform t1 X Coefficients")

fig4.show()

In [ ]:
""" coef1 permutation entropy"""
t1xcoef1PE = []
for i in range(0,len(t1xcoefs[0]), 128):
    t1xcoef1PE.append(perm_entropy(t1xcoefs[0][i:i+128], normalize=True))

In [ ]:
"""Moving Window Average for coef1 Permutation Entropy"""
t1xavg_coef1PE = []
for i in range(0, len(t1xcoef1PE)-56, 1):
    t1xavg_coef1PE.append(mean(t1xcoef1PE[i:i+56]))

In [ ]:
""" coef2 permutation entropy"""
t1xcoef2PE = []
for i in range(0,len(t1xcoefs[1]), 128):
    t1xcoef2PE.append(perm_entropy(t1xcoefs[1][i:i+128], normalize=True))

In [ ]:
"""Moving Window Average for coef2 Permutation Entropy"""
t1xavg_coef2PE = []
for i in range(0, len(t1xcoef2PE)-56, 1):
    t1xavg_coef2PE.append(mean(t1xcoef2PE[i:i+56]))

In [ ]:
""" coef3 permutation entropy"""
t1xcoef3PE = []
for i in range(0,len(t1xcoefs[2]), 128):
    t1xcoef3PE.append(perm_entropy(t1xcoefs[2][i:i+128], normalize=True))

In [ ]:
"""Moving Window Average for coef3 Permutation Entropy"""
t1xavg_coef3PE = []
for i in range(0, len(t1xcoef3PE)-56, 1):
    t1xavg_coef3PE.append(mean(t1xcoef3PE[i:i+56]))

In [ ]:
""" coef4 permutation entropy"""
t1xcoef4PE = []
for i in range(0,len(t1xcoefs[3]), 128):
    t1xcoef4PE.append(perm_entropy(t1xcoefs[3][i:i+128], normalize=True))

In [ ]:
"""Moving Window Average for cD4 Permutation Entropy"""
t1xavg_coef4PE = []
for i in range(0, len(t1xcoef4PE)-56, 1):
    t1xavg_coef4PE.append(mean(t1xcoef4PE[i:i+56]))

In [ ]:
""" coef5 permutation entropy"""
t1xcoef5PE = []
for i in range(0,len(t1xcoefs[4]), 128):
    t1xcoef5PE.append(perm_entropy(t1xcoefs[4][i:i+128], normalize=True))

In [ ]:
"""Moving Window Average for cD5 Permutation Entropy"""
t1xavg_coef5PE = []
for i in range(0, len(t1xcoef5PE)-56, 1):
    t1xavg_coef5PE.append(mean(t1xcoef5PE[i:i+56]))

In [ ]:
"""Matrix of Continuous Wavelet Transform coefficients Permutation Entropy"""

fig5 = make_subplots(rows = 2, cols = 3, start_cell = 'top-left', subplot_titles=("Avg coef1 PE","Avg coef2 PE","Avg coef3 PE","Avg coef4 PE","Avg coef5 PE", "BIS"))
fig5.append_trace(go.Line(y = t1xavg_coef1PE, name = 'Coef1 PE'), 1,1)
fig5.append_trace(go.Line(y = t1xavg_coef2PE, name = 'Coef2 PE'), 1,2)
fig5.append_trace(go.Line(y = t1xavg_coef3PE, name = 'Coef3 PE'), 1,3)
fig5.append_trace(go.Line(y = t1xavg_coef4PE, name = 'Coef4 PE'), 2,1)
fig5.append_trace(go.Line(y = t1xavg_coef5PE, name = 'Coef5 PE'), 2,2)
fig5.append_trace(go.Line(y = t1.BIS[0:3535:], name = 'BIS'), 2,3)

fig5.update_layout(title="Continuous Wavelet Transform t1 X Coefficients 56 Sec Window Average Perm Entropy")

fig5.show()

In [ ]:
"""CALCULATING THE MOVING WINDOW AVERAGE Higuchi Fractal Dimension FOR THE CH2 CWT SUB-BANDS"""

In [ ]:
"""coef1 Higuchi Fractal Dimension"""
t1xcoef1HFD = []
for i in range(0,len(t1xcoefs[0]), 128):
    t1xcoef1HFD.append(higuchi_fd(t1xcoefs[0][i:i+128], kmax = 10))

In [ ]:
"""Moving Window Average for coef1 Higuchi Fractal Dimension"""
t1xavg_coef1HFD = []
for i in range(0, len(t1xcoef1HFD)-56, 1):
    t1xavg_coef1HFD.append(mean(t1xcoef1HFD[i:i+56]))

In [ ]:
"""coef2 Higuchi Fractal Dimension"""
t1xcoef2HFD = []
for i in range(0,len(t1xcoefs[1]), 128):
    t1xcoef2HFD.append(higuchi_fd(t1xcoefs[1][i:i+128], kmax = 10))

In [ ]:
"""Moving Window Average for coef2 Higuchi Fractal Dimension"""
t1xavg_coef2HFD = []
for i in range(0, len(t1xcoef2HFD)-56, 1):
    t1xavg_coef2HFD.append(mean(t1xcoef2HFD[i:i+56]))

In [ ]:
"""coef3 Higuchi Fractal Dimension"""
t1xcoef3HFD = []
for i in range(0,len(t1xcoefs[2]), 128):
    t1xcoef3HFD.append(higuchi_fd(t1xcoefs[2][i:i+128], kmax = 10))

In [ ]:
"""Moving Window Average for coef3 Higuchi Fractal Dimension"""
t1xavg_coef3HFD = []
for i in range(0, len(t1xcoef3HFD)-56, 1):
    t1xavg_coef3HFD.append(mean(t1xcoef3HFD[i:i+56]))

In [ ]:
"""coef4 Higuchi Fractal Dimension"""
t1xcoef4HFD = []
for i in range(0,len(t1xcoefs[3]), 128):
    t1xcoef4HFD.append(higuchi_fd(t1xcoefs[3][i:i+128], kmax = 10))

In [ ]:
"""Moving Window Average for coef4 Higuchi Fractal Dimension"""
t1xavg_coef4HFD = []
for i in range(0, len(t1xcoef4HFD)-56, 1):
    t1xavg_coef4HFD.append(mean(t1xcoef4HFD[i:i+56]))

In [ ]:
"""coef5 Higuchi Fractal Dimension"""
t1xcoef5HFD = []
for i in range(0,len(t1xcoefs[4]), 128):
    t1xcoef5HFD.append(higuchi_fd(t1xcoefs[4][i:i+128], kmax = 10))

In [ ]:
"""Moving Window Average for coef5 Higuchi Fractal Dimension"""
t1xavg_coef5HFD = []
for i in range(0, len(t1xcoef5HFD)-56, 1):
    t1xavg_coef5HFD.append(mean(t1xcoef5HFD[i:i+56]))

In [ ]:
"""Matrix of Continuous Wavelet Transform CH2 coefficients Higuchi Fractal Dimension"""

fig6 = make_subplots(rows = 2, cols = 3, start_cell = 'top-left', subplot_titles=("Avg coef1 HFD","Avg coef2 HFD","Avg coef3 HFD","Avg coef4 HFD","Avg coef5 HFD", "BIS"))
fig6.append_trace(go.Line(y = t1xavg_coef1HFD, name = 'Coef1 HFD'), 1,1)
fig6.append_trace(go.Line(y = t1xavg_coef2HFD, name = 'Coef2 HFD'), 1,2)
fig6.append_trace(go.Line(y = t1xavg_coef3HFD, name = 'Coef3 HFD'), 1,3)
fig6.append_trace(go.Line(y = t1xavg_coef4HFD, name = 'Coef4 HFD'), 2,1)
fig6.append_trace(go.Line(y = t1xavg_coef5HFD, name = 'Coef5 HFD'), 2,2)
fig6.append_trace(go.Line(y = t1.BIS[0:3535:], name = 'BIS'), 2,3)

fig6.update_layout(title="Continuous Wavelet Transform t1 X Coefficients 56 Sec Window Average Higuchi Fractal Dimension")

fig6.show()

In [ ]:
"""WRITING FEATURES TO XLSX"""
import xlsxwriter

In [ ]:
df = pd.DataFrame(t1avg_coef4PE)
df['c4HFD'] = t1avg_coef4HFD
df['c3PE'] = t1avg_coef3PE
df['c3HFD'] = t1avg_coef3HFD
df['BIS'] = t1.BIS[0:3535:]

In [ ]:
df.rename(columns = {0:'c4PE'}, inplace = True)

In [ ]:
df.head()

In [ ]:
df = pd.DataFrame(t1avg_coef4PE)
df['c4HFD'] = t1avg_coef4HFD
df['c3PE'] = t1avg_coef3PE
df['c3HFD'] = t1avg_coef3HFD
df['BIS'] = t1.BIS[0:3535:]

df.rename(columns = {0:'c4PE'}, inplace = True)

df.head()

df.to_excel("Case1.xlsx", sheet_name='Case1', engine = 'xlsxwriter')